In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import re
import dask.dataframe as dd
root = 'data/'

In [2]:
from geopandas import GeoDataFrame
from geopandas import sjoin
from shapely.geometry import Point
from shapely.geometry import LineString
from descartes.patch import PolygonPatch

# LOAD TRANSIT STATION DATA

Source : NYC MTA (Subway Stations Data)

Description : Description of all the subway stations in NYC. Useful fields are 'STOP_NAME' (station name) and 'GTFS Latitude' and 'GTFS Longitude' (geographic coordinates of the station)

Processing : This dataset has been processed by the stations.ipynb python notebook and saved to Stations_geomerged.geojson

Following data issues have been addressed :

1. There is no unique identifier that represents stations across the NYC MTA database : The 'STATION' column of the MTA turnstile dataset is the only identifier for the station in that set. The contents of this column differed significantly from the 'STOP_NAME' column of the stations dataset. For example, the station named 'TIMES SQ-42 ST' in one set was represented as 'Times Sq - 42 St' in the other. Although, issues like this were easy to fix, there were a fair number of cases where a station named 'Astoria - Ditmars Blvd' did not have any obvious match in the other data set. A possible cause for cases like this is the use of different station names for the same station ('Astoria - Ditmars Blvd' station was earlier known as 'Second Avenue'). Cases like this are hard, if not impossible to match. A python string-matching library called 'fuzzy-wuzzy' was used to find the best matches using 3 Levenshtein closeness ratios (normal ratio, partial ratio and token sort ratio). The match was accepted only if one of the three matching methods returned a ratio of 88% or higher. The matching station names from the trunstile dataset were added to the 'STATION' column of the stations dataset.

    
2. The columns 'GTFS Latitude' and 'GTFS Longitude' required further processing in order to be readily consumable for joins (geographic) across different datasets (for example with the traffic and taxi/cab datasets) : The python geopandas library was leveraged for this purpose (This library in turn depends on shapely, fiona and rtree). 'GTFS Latitude' and 'GTFS Longitude' were merged into a single 'Point' geometry (shapely.geometry.Point) and the entire datset converted to a geopandas GeoDataFrame. This allows for fairly easy (though sometimes computationally expensive) joins across datasets using the geometry attributes like, points, lines and polygons. A circle of customizable radius, centered at each station, was also drawn and added to a new geometry column containing the circles as polygons. These circles represent 'circles of influence' or zones for each station and will be used to find intersection with traffic and taxi/cab data.

In [3]:
file = root + 'transit/Stations_geomerged.geojson'
geodf_stations = GeoDataFrame.from_file(file)
geodf_stations.head()

,STATION_ID,STOP_ID,STOP_NAME,BOROUGH,STATION,geometry
0,1,R01,Astoria - Ditmars Blvd,Q,None,"POLYGON ((40.785036 -73.91203400000001, 40.784..."
1,2,R03,Astoria Blvd,Q,ASTORIA BLVD,"POLYGON ((40.780258 -73.917843, 40.78020984726..."
2,3,R04,30 Av,Q,30 AV,"POLYGON ((40.776779 -73.92147900000001, 40.776..."
3,4,R05,Broadway,Q,BROADWAY,"POLYGON ((40.77182 -73.92550799999999, 40.7717..."
4,5,R06,36 Av,Q,36 AV,"POLYGON ((40.766804 -73.929575, 40.76675584726..."


# LOAD AND CLEAN TRANSIT DATA

Source : NYC MTA (Subway Stations Turnstile Data : 4-hour frequency, 2016 and 2017)

Description : Transit ridership (turnstile entry and exit counts) of all the subway stations in NYC. Useful fields are 'STATION' (station name), 'DATE' (date) , 'TIME' (time) , 'ENTRIES' (entry count) , 'EXITS' (exit count)

Processing : 

Following data issues have been addressed :

1. 'DATE' and 'TIME' occur as separate string columns : These two were merged and converted to type 'datetime64[ns]'. This column was also used as the index (after the rest of cleaning was complete)


2. 'EXITS' (and 'ENTRIES') columns have cumulative reading of the turnstile unit : the pandas.Series.diff method was used to calculate the change from the previous reading. 


3. Turnstile units would reset randomly once in a while, resulting in outliers in the 'EXITS' (and 'ENTRIES') columns (abmormally high values or negative values) : These outliers were identified and filtered out by calculating the inter-quartile range and rejecting all rows with 'EXITS' (or 'ENTRIES') with values greater that 5 times the inter-quartile range or with negative values. 

In [4]:
#taking 2 arguments date and time instead of single datetime improved performance drastically
def parse_date(date,time):
    return pd.to_datetime(date+time,format="%m/%d/%Y%H:%M:%S", errors='coerce')
def parse_int(exits):
    return pd.to_numeric(exits,errors='coerce')

In [5]:
col_func = lambda x:x.strip().upper() in ['STATION','DATE','TIME','EXITS']

In [6]:
#consider exits and entries both or just one of them? and why?
file = root + 'transit/all_turnstile_1617.txt'
transit_df = pd.read_csv(file,header=0,parse_dates={'DATETIME': ['DATE','TIME']},
                         usecols = col_func,skipinitialspace=True, low_memory=False,
                         date_parser=parse_date)

In [7]:
transit_df.info()
transit_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20526873 entries, 0 to 20526872
Data columns (total 3 columns):
DATETIME                                                                datetime64[ns]
STATION                                                                 object
EXITS                                                                   object
dtypes: datetime64[ns](1), object(2)
memory usage: 469.8+ MB


,DATETIME,STATION,EXITS
0,2015-12-26 03:00:00,59 ST,0001846630 ...
1,2015-12-26 07:00:00,59 ST,0001846640 ...
2,2015-12-26 11:00:00,59 ST,0001846739 ...
3,2015-12-26 15:00:00,59 ST,0001846805 ...
4,2015-12-26 19:00:00,59 ST,0001846874 ...


In [8]:
transit_df = transit_df.rename(columns=lambda x: x.strip())

#read multiple files in a loop into dataframes and then concat them

In [9]:
transit_df = transit_df.drop_duplicates()
transit_df = transit_df.dropna()

In [10]:
transit_df['EXITS'] = pd.to_numeric(transit_df['EXITS'],errors='coerce')
transit_df = transit_df.set_index('DATETIME')
transit_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20420523 entries, 2015-12-26 03:00:00 to 2017-12-29 20:00:00
Data columns (total 2 columns):
STATION    object
EXITS      int64
dtypes: int64(1), object(1)
memory usage: 467.4+ MB


In [11]:
transit_df['DELEXITS']= transit_df['EXITS'].diff()
#transit_df['EXITS'] = transit_df.apply(fix_exits,axis=1,iqr=intqrange)

In [12]:
transit_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20420523 entries, 2015-12-26 03:00:00 to 2017-12-29 20:00:00
Data columns (total 3 columns):
STATION     object
EXITS       int64
DELEXITS    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 623.2+ MB


In [13]:
intqrange = transit_df['DELEXITS'].quantile(0.75) - transit_df['DELEXITS'].quantile(0.25)
discard = (transit_df['DELEXITS'] < 0) | (transit_df['DELEXITS'] > 5*intqrange)
transit_df = transit_df.loc[~discard]
transit_df = transit_df.dropna()

In [14]:
transit_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 19364090 entries, 2015-12-26 07:00:00 to 2017-12-29 16:00:00
Data columns (total 3 columns):
STATION     object
EXITS       int64
DELEXITS    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 590.9+ MB


In [15]:
transit_df.describe()

,EXITS,DELEXITS
count,1.936409e+07,1.936409e+07
mean,2.246338e+07,1.128584e+02
std,1.501158e+08,1.529398e+02
min,0.000000e+00,0.000000e+00
25%,2.623010e+05,8.000000e+00
50%,1.444191e+06,5.000000e+01
75%,4.487752e+06,1.540000e+02
max,2.140537e+09,8.100000e+02


# LOAD CAB DATA

Source : NYC TLC (Taxi and Cab Trip Data : every taxi/cab trip in NYC for 2016 and 2017)

Description : Every taxi/cab trip in NYC. Useful fields are 'dropoff_datetime', 'dropoff_latitude', 'dropoff_longitude' , 'pickup_datetime', 'pickup_latitude', 'pickup_longitude', 'passenger_count'

Processing :

Following data issues have been addressed :

1. The dataset for 2016 and 2017 is too large and called for parallel processing techniques : The python API Dask was leveraged for this. This partitions large datsets into multiple pandas DataFrames and allows for parallel processing on them.


2. The columns 'dropoff_latitude', 'dropoff_longitude' (and 'pickup_latitude', 'pickup_longitude') required further processing in order to be readily consumable for joins (geographic) with the Stations dataset : The python geopandas library was leveraged for this purpose (This library in turn depends on shapely, fiona and rtree). 'dropoff_latitude', 'dropoff_longitude' were merged into a single 'Point' geometry (shapely.geometry.Point) and the entire datset converted to a geopandas GeoDataFrame. This allows for fairly easy (though computationally expensive in this case due to the size of the dataset) joins across datasets using the geometry attributes like, points, lines and polygons. A circle of customizable radius, centered at each station, representing the 'circles of influence' or zones for each station will be used to find intersection with taxi/cab data. (each trip will be associated with a station for the pickup point, as well as a station for the dropoff point, by finding which station-zone the points fall in)

    
The processed data is saved in parquet format (processed and saved by the cabs notebook), to enable quick reading by dask in the clean_and_wrangle notebook

In [ ]:
cabs_df = dd.read_parquet(root+'cabs/geojoined')
cabs_df.info()
cabs_df.head()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, passenger_count to STATION
dtypes: object(1), float64(2), int64(1)

# LOAD TRAFFIC LINK DATA

Source : NYC Open data (Traffic Links Data)

Description : Traffic Links (sets of geographic coordinates) each representing a stretch or road/street over which the average speed of traffic is recorded. Useful fields are 'LINK_ID','LINK_POINTS','BOROUGH'

Processing :

Following data issues have been addressed :

1. The column 'LINK_POINTS' required further processing in order to be readily consumable for joins (geographic) with the Stations dataset : The python geopandas library was leveraged for this purpose (This library in turn depends on shapely, fiona and rtree). The contents of 'LINK_POINTS' were merged into a single 'LineString' geometry (shapely.geometry.LineString) and the entire datset converted to a geopandas GeoDataFrame. This allows for fairly easy joins across datasets using the geometry attributes like points, lines and polygons. 


2. Association of each 'LINK_ID' with a Station : A circle of customizable radius, centered at each station, representing the 'circles of influence' or zones for each station (represented as a Polygon geometry in the Stations dataset) was used to find intersection with traffic link data. (each link was associated with a station by finding which station-zone the link-line intersects with)

    
The processed data is saved in geojson format, to enable quick reading in the clean_and_wrangle notebook

In [2]:
file = root + 'traffic/Traffic_Links_geomerged.geojson'
geodf_traffic_links = GeoDataFrame.from_file(file)
geodf_traffic_links.head()

NameError: name 'root' is not defined

# LOAD AND CLEAN TRAFFIC DATA

Source : NYC Open data (Traffic speed data Data recorded at various locations in NYC for 2016 and 2017)

Description : Traffic speed data Data recorded at various locations in NYC. Useful fields are 'LINK_ID','DATETIME','SPEED'

Processing :

No issues found with this set so far.

In [99]:
file = root + 'traffic/DOT_Traffic_1617_hourly.csv'
df_traffic = pd.read_csv(file, header=None,parse_dates=[1])
df_traffic.columns = ['LINK_ID','DATETIME','SPEED']
#index by datetime
df_traffic = df_traffic.set_index('DATETIME')
df_traffic.info()
df_traffic.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 923481 entries, 2017-05-05 09:00:00 to 2017-12-31 23:00:00
Data columns (total 2 columns):
LINK_ID    923481 non-null int64
SPEED      511579 non-null float64
dtypes: float64(1), int64(1)
memory usage: 21.1 MB


,LINK_ID,SPEED
DATETIME,,
2017-05-05 09:00:00,4329472,8.493333
2017-05-05 10:00:00,4329472,7.313846
2017-05-05 11:00:00,4329472,22.446250
2017-05-05 12:00:00,4329472,28.640909
2017-05-05 13:00:00,4329472,25.414000


# LOAD AND CLEAN WEATHER DATA

Source : National Climatic Data Center (daily temperature, rainfall and snowfall data for NYC for 2016 and 2017)

Description : Daily temperature, rainfall and snowfall data for NYC. Useful fields are 'DATE','PRCP','SNOW','TMAX','TMIN'

Processing :

Following data issues have been addressed :

1. 'DATE' in string format : This was converted to type 'datetime64[ns]'. This column was also used as the index (after the rest of cleaning was complete)


2. Average daily temperature : This was calculated by finding the mean of 'TMIN' and 'TMAX' and added as 'TAVG'

In [95]:
def parse_date_3(date):
    return pd.to_datetime(date,format="%Y-%m-%d", errors='coerce')

In [96]:
#weather data
file = root + 'weather/1409973.csv'
weather_df = pd.read_csv(file,header=0,parse_dates=['DATE'],
                         usecols=['DATE','PRCP','SNOW','TMAX','TMIN'],
                         skipinitialspace=True,
                         date_parser=parse_date_3)
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2027 entries, 0 to 2026
Data columns (total 5 columns):
DATE    2027 non-null datetime64[ns]
PRCP    2027 non-null float64
SNOW    2027 non-null float64
TMAX    2027 non-null int64
TMIN    2027 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 79.3 KB


In [97]:
weather_df = weather_df.dropna().set_index('DATE')
weather_df = weather_df.loc['2016-01-01':'2017-12-31']
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 731 entries, 2016-01-01 to 2017-12-31
Data columns (total 4 columns):
PRCP    731 non-null float64
SNOW    731 non-null float64
TMAX    731 non-null int64
TMIN    731 non-null int64
dtypes: float64(2), int64(2)
memory usage: 28.6 KB


In [98]:
weather_df['TAVG'] = (weather_df['TMIN'] + weather_df['TMAX']) / 2

# LOAD AND CLEAN GAS PRICE DATA

Source : https://www.nyserda.ny.gov (monthly gas prices for NYC for 2016 and 2017)

Description : Monthly gas prices for NYC.

Processing :

Following data issues have been addressed :

1. Each year is a different column : The dataframe was melted to create a single column called 'YEAR'. The 'MONTH' and 'YEAR' were then combined and converted to datetime and used as index.

In [124]:
file = root + 'gas/2018-2008_monthly_gas_NYC.csv'
gas_df = pd.read_csv(file, header=0, skipinitialspace=True)
gas_df.info()
gas_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 12 columns):
Unnamed: 0    12 non-null object
2018          6 non-null float64
2017          12 non-null float64
2016          12 non-null float64
2015          12 non-null float64
2014          12 non-null float64
2013          12 non-null float64
2012          12 non-null float64
2011          12 non-null float64
2010          12 non-null float64
2009          12 non-null float64
2008          12 non-null float64
dtypes: float64(11), object(1)
memory usage: 1.2+ KB


,Unnamed: 0,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
0,Jan,261.1,245.6,194.5,226.4,347.7,353.7,348.4,318.9,273.9,173.3,309.2
1,Feb,267.0,238.7,178.0,222.5,347.1,376.4,368.9,327.4,270.9,191.1,303.8
2,Mar,260.2,233.6,184.7,240.3,359.1,374.1,383.0,358.4,274.9,193.0,318.4
3,Apr,277.4,241.7,206.1,244.4,366.3,358.9,394.0,381.3,282.0,201.9,339.8
4,May,297.0,242.0,223.2,268.0,372.9,357.7,378.1,402.1,290.0,226.1,381.2
5,Jun,293.8,239.3,228.4,275.3,375.2,359.5,355.6,381.3,277.7,262.9,412.7
6,Jul,NaN,235.6,222.2,275.8,371.7,369.2,355.9,378.2,276.0,260.2,413.7
7,Aug,NaN,242.8,213.7,254.3,353.8,371.5,379.0,377.3,272.0,266.7,382.8
8,Sep,NaN,276.1,215.7,227.0,342.5,363.9,392.3,369.7,264.8,262.2,360.6
9,Oct,NaN,255.8,218.5,214.8,323.6,341.5,386.3,352.6,281.2,254.9,305.8


In [125]:
gas_df = gas_df.rename(columns={'Unnamed: 0':'MONTH'})
gas_df.columns

Index(['MONTH', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011',
       '2010', '2009', '2008'],
      dtype='object')

In [126]:
gas_df = gas_df.melt(id_vars=['MONTH'],var_name='YEAR',value_name='PRICE')
gas_df.head()

,MONTH,YEAR,PRICE
0,Jan,2018,261.1
1,Feb,2018,267.0
2,Mar,2018,260.2
3,Apr,2018,277.4
4,May,2018,297.0


In [127]:
gas_df['MONTH'] = pd.to_datetime(gas_df['MONTH']+'-'+gas_df['YEAR'],format='%b-%Y')
del gas_df['YEAR']

gas_df.head()

,MONTH,PRICE
0,2018-01-01,261.1
1,2018-02-01,267.0
2,2018-03-01,260.2
3,2018-04-01,277.4
4,2018-05-01,297.0


In [128]:
gas_df = gas_df.set_index('MONTH')
gas_df.head()

,PRICE
MONTH,
2018-01-01,261.1
2018-02-01,267.0
2018-03-01,260.2
2018-04-01,277.4
2018-05-01,297.0


In [129]:
gas_df = gas_df.sort_index().loc['2016-01-01':'2017-12-31']
gas_df.head()

,PRICE
MONTH,
2016-01-01,194.5
2016-02-01,178.0
2016-03-01,184.7
2016-04-01,206.1
2016-05-01,223.2


# FILTER ALL TIME SERIES DATASETS BY DAY OF WEEK AND RESAMPLE

Wrangling of each of the time series datasets created/loaded above :

1. Filter by day of the week : The time-series datasets for transit, traffic, cabs and weather were filtered by day of the week (this is because of the fact that datasets like transit and traffic, for instance, would have different patterns on weekdays than on weekends and it therefore makes sense to seperate out and compare trends by day of the week)

    
2. Aggregate the filtered data over 1 month window : All time-series datasets (except gas which is already aggregated by month), were resampled using frequency = '1M' and aggregated using mean, median or sum functions

In [1]:
#week day
weekday = 2

transit_df_byday = transit_df_merged.loc[transit_df_merged.index.weekday==weekday]
transit_df_byday.info()
transit_df_byday.head()

NameError: name 'transit_df_merged' is not defined

In [115]:
#transit_df_rsmpld = transit_df.reset_index().set_index('DATETIME').resample('1D',how='sum')
transit_df_rsmpld = transit_df_byday.reset_index().groupby('FUZZY_STATION').apply(lambda x: x.set_index('DATETIME').resample('1M').sum()).swaplevel(1,0)
transit_df_rsmpld.info()
transit_df_rsmpld.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7915 entries, (2015-12-31 00:00:00, 1 Av) to (2017-12-31 00:00:00, Zerega Av)
Data columns (total 2 columns):
EXITS       7915 non-null int64
DELEXITS    7915 non-null float64
dtypes: float64(1), int64(1)
memory usage: 149.7+ KB


,,EXITS,DELEXITS
DATETIME,FUZZY_STATION,,
2015-12-31,1 Av,8713408891,12187.0
2016-01-31,1 Av,31723420322,41172.0
2016-02-29,1 Av,31768845053,42989.0
2016-03-31,1 Av,42124791060,48218.0
2016-04-30,1 Av,34636336475,40406.0


In [116]:
idx = pd.IndexSlice
transit_df_rsmpld = transit_df_rsmpld.sort_index().loc[idx['2016-01-01':'2017-12-31',:],:].sort_index()
transit_df_rsmpld.info()
transit_df_rsmpld.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7600 entries, (2016-01-31 00:00:00, 1 Av) to (2017-12-31 00:00:00, Zerega Av)
Data columns (total 2 columns):
EXITS       7600 non-null int64
DELEXITS    7600 non-null float64
dtypes: float64(1), int64(1)
memory usage: 143.8+ KB


EXITS  DELEXITS
DATETIME   FUZZY_STATION                       
2016-01-31 1 Av           31723420322   41172.0
           103 St         19714566035  120605.0
           104 St         25959775845    7565.0
           110 St           619844014   35514.0
           111 St          1295305410   36746.0

In [143]:
#weekday = 2
traffic_df = df_traffic.dropna()
traffic_df_byday = traffic_df.loc[traffic_df.index.weekday==weekday]
traffic_df_byday.info()
traffic_df_byday.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 78274 entries, 2017-05-17 07:00:00 to 2017-12-27 23:00:00
Data columns (total 2 columns):
LINK_ID    78274 non-null int64
SPEED      78274 non-null float64
dtypes: float64(1), int64(1)
memory usage: 1.8 MB


,LINK_ID,SPEED
DATETIME,,
2017-05-17 07:00:00,4329472,12.426667
2017-05-17 08:00:00,4329472,10.097500
2017-05-17 09:00:00,4329472,8.813636
2017-05-17 10:00:00,4329472,6.700000
2017-05-17 11:00:00,4329472,7.740000


In [144]:
traffic_df_rsmpld = traffic_df_byday.reset_index().groupby('LINK_ID').apply(lambda x: x.set_index('DATETIME').resample('1M').median()).swaplevel(1,0)
traffic_df_rsmpld.info()
traffic_df_rsmpld.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1202 entries, (2017-05-31 00:00:00, 4329472) to (2017-12-31 00:00:00, 4763657)
Data columns (total 2 columns):
LINK_ID    1054 non-null float64
SPEED      1054 non-null float64
dtypes: float64(2)
memory usage: 23.6 KB


,,LINK_ID,SPEED
DATETIME,LINK_ID,,
2017-05-31,4329472,4329472.0,13.952727
2017-06-30,4329472,4329472.0,13.384615
2017-07-31,4329472,4329472.0,15.759091
2017-08-31,4329472,4329472.0,25.525417
2017-09-30,4329472,4329472.0,23.373750


In [145]:
#idx = pd.IndexSlice
traffic_df_rsmpld = traffic_df_rsmpld.sort_index().loc[idx['2016-01-01':'2017-12-31',:],:].sort_index()
traffic_df_rsmpld.info()
traffic_df_rsmpld.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1202 entries, (2016-09-30 00:00:00, 4616192) to (2017-12-31 00:00:00, 4763657)
Data columns (total 2 columns):
LINK_ID    1054 non-null float64
SPEED      1054 non-null float64
dtypes: float64(2)
memory usage: 23.6 KB


LINK_ID  SPEED
DATETIME   LINK_ID                  
2016-09-30 4616192  4616192.0  45.98
2016-10-31 4456516  4456516.0  14.29
           4616192        NaN    NaN
           4616193  4616193.0   4.97
           4616194  4616194.0   6.21

In [163]:
#resample transit and cab data to a lower frequency (1 Month)

#select by day
cabs_df_byday = cabs_df.loc[cabs_df.index.weekday == weekday]
cabs_df_byday.info()
cabs_df_byday.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 39282163 entries, 2016-01-06 00:00:00 to 2017-12-27 23:59:59
Data columns (total 1 columns):
passenger_count    object
dtypes: object(1)
memory usage: 599.4+ MB


,passenger_count
dropoff_datetime,
2016-01-06,2
2016-01-06,1
2016-01-06,1
2016-01-06,1
2016-01-06,1


In [164]:
cabs_df_rsmpld = cabs_df_byday.resample('1M')['passenger_count'].count()
#transit_df = transit_df.reset_index().set_index('DATETIME')
#transit_df_rsmpld = transit_df_rsmpld.reset_index().set_index(['DATETIME','STATION'])
cabs_df_rsmpld.head()

dropoff_datetime
2016-01-31    1660372
2016-02-29    1740040
2016-03-31    2185287
2016-04-30    1742900
2016-05-31    1734233
Freq: M, Name: passenger_count, dtype: int64

In [120]:
#select by day
weather_df_byday = weather_df.loc[weather_df.index.weekday == weekday]
weather_df_byday.info()
weather_df_byday.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 104 entries, 2016-01-06 to 2017-12-27
Data columns (total 5 columns):
PRCP    104 non-null float64
SNOW    104 non-null float64
TMAX    104 non-null int64
TMIN    104 non-null int64
TAVG    104 non-null float64
dtypes: float64(3), int64(2)
memory usage: 4.9 KB


,PRCP,SNOW,TMAX,TMIN,TAVG
DATE,,,,,
2016-01-06,0.00,0.0,41,25,33.0
2016-01-13,0.00,0.0,30,22,26.0
2016-01-20,0.00,0.0,37,27,32.0
2016-01-27,0.00,0.0,47,34,40.5
2016-02-03,0.73,0.0,59,42,50.5


In [121]:
weather_df_rsmpld = weather_df_byday.resample('1M').mean()
#weather_snow = weather_df['SNOW'].resample('1M').mean()
#weather_snow = weather_df['SNOW'].resample('1M').mean()

weather_df_rsmpld.head()

,PRCP,SNOW,TMAX,TMIN,TAVG
DATE,,,,,
2016-01-31,0.0000,0.0,38.75,27.00,32.875
2016-02-29,0.4900,0.0,49.25,36.00,42.625
2016-03-31,0.0320,0.0,64.80,41.20,53.000
2016-04-30,0.0000,0.0,59.25,41.25,50.250
2016-05-31,0.0025,0.0,71.00,52.75,61.875
